In [97]:
#importing the libraries
import pandas as pd
import numpy as np
import re

In [98]:
#reading the data
raw_data = pd.read_csv(r'naive_data.csv')
print(raw_data)

   Great product. Have been using it for more than 1.5 years now. No issues  \
0   The protection level is poor. My phone fell of...                         
1                        Love the blue bumper case.+1                         
2   This is a great band! Never used one before bu...                         
3                              Never buy this product                         
4   I love the shoes. Thess are my first lotto sho...                         
5            Good product. Happy with the purchase.+1                         
6              Product is great. Same as described.+1                         
7                               Very poor quality.+-1                         
8                          Nice kurta rich looking.+1                         
9                                        Not good.+-1                         
10  T-Shirt is different in real than display pic....                         
11  very poor quality shoes.my left heel has state..

In [111]:
#analysing the data
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 2 columns):
 #   Column                                                                                              Non-Null Count  Dtype 
---  ------                                                                                              --------------  ----- 
 0   Great product. Have been using it for more than 1.5 years now. No issues                            20 non-null     object
 1    no hanging problem. Battery life is same as on day one. Each of their products is just awesome.+1  3 non-null      object
dtypes: object(2)
memory usage: 448.0+ bytes


In [100]:
df=open("naive_data.csv","r")

reader = csv.reader(df, delimiter='+')

In [101]:
# It is the dictionary that has the data : { label(positive/negative) : { word : count of number of occurrences of the word } }

dataset={}

In [102]:
# The dictionary with count of records, { label l : No. of records that are labeled l }

no_of_items={}

In [103]:
# The dictionary with count of occurrances of word under each label

feature_set={}

In [104]:
# For each sentence in dataset

for row in reader:
    # Initialize the label in the dictionary if not present already
    no_of_items.setdefault(row[1],0)
    
    # Increase the count of occurence of label by 1 for every occurence
    no_of_items[row[1]]+=1
    
    # Initialize the dictionary for a label if not present
    dataset.setdefault(row[1],{})
    
    # Split the sentence with respect to non-characters, and donot split if apostophe is present
    split_data=re.split('[^a-zA-Z\']',row[0])
    
    # For every word in split data
    for i in split_data:
        # Removing stop words to a small extent by ignoring words with length less than 3
        if len(i) > 2:
            dataset[row[1]].setdefault(i.lower(),0)
            
            # Increase the word count on its occurrence with label row[1]
            dataset[row[1]][i.lower()]+=1
            
            # Initialize a dictionary for a newly found word in feature set
            feature_set.setdefault(i.lower(),{})
            
            # If the label was found for the word, for the first time, initialize corresponding count value for word as key
            feature_set[i.lower()].setdefault(row[1],0)
            
            # Increment the count for the word in that label
            feature_set[i.lower()][row[1]]+=1


In [105]:
# To calculate the basic probability of a word for a category

def calc_prob(word,category):
    if word not in feature_set or word not in dataset[category]:
        return 0

    return float(dataset[category][word])/no_of_items[category]

In [106]:
# Weighted probability of a word for a category

def weighted_prob(word,category):
    basic_prob=calc_prob(word,category)
    if word in feature_set:
        tot=sum(feature_set[word].values())
    else:
        tot=0
    weight_prob=((1.0*0.5)+(tot*basic_prob))/(1.0+tot)
    return weight_prob


In [107]:
# To get probability of the test data for the given category

def test_prob(test,category):
    split_data=re.split('[^a-zA-Z][\'][ ]',test)
    data=[]
    for i in split_data:
        if ' ' in i:
            i=i.split(' ')
            for j in i:
                if j not in data:
                    data.append(j.lower())
        elif len(i) > 2 and i not in data:
            data.append(i.lower())
            
    p=1
    for i in data:
        p*=weighted_prob(i,category)
    return p
  
        
    

In [108]:
# Naive Bayes implementation

def naive_bayes(test):
    results={}
    for i in dataset.keys():
        # Category Probability
        # Number of items in category/total number of items
        cat_prob=float(no_of_items[i])/sum(no_of_items.values())
        test_prob1=test_prob(test,i)
        
        results[i]=test_prob1*cat_prob
    return results

In [112]:
# Predicting the final output

print ('Enter the sentence')
text=input('Sentence = ')
result=naive_bayes(text)

if result['1'] > result['-1']:
    print ('The Sentence is : positive(+ve)')
else:
    print ('The Sentence is : negative(-ve)')


Enter the sentence
Sentence = It was the bad movie I've seen
The Sentence is : negative(-ve)
